In [1]:
# Import library
import pandas as pd
import sqlite3

In [2]:
# Données du Guide de l'auto
df = pd.read_hdf('C:\\Users\\Charles_tour\\Documents\\GitHub\\car_sales_forecast\\Dataframe\\scrap_data.h5')
df.head(3)

,marque,modele,annee,spec,pdsf,moteur,type_carbu,type_vehicule,classement,eff_ville,eff_autoroute,eff_combi,eff_autonomie,eff_ges,masse,lien
0,acura,integra,2024,a-spec,38850$,"4L 1,5l",Super,Hatchback,Sous-compacte de luxe,"8,1L/100km","6,5L/100km","7,4L/100km",621km,172g/km,1415kg (3120lb),https://www.guideautoweb.com/constructeurs/acu...
1,acura,integra,2024,elite-a-spec-auto,44350$,"4L 1,5l",Super,Hatchback,Sous-compacte de luxe,"8,1L/100km","6,5L/100km","7,4L/100km",621km,172g/km,1434kg (3161lb),https://www.guideautoweb.com/constructeurs/acu...
2,acura,integra,2024,elite-a-spec-man,44350$,"4L 1,5l",Super,Hatchback,Sous-compacte de luxe,"8,9L/100km","6,5L/100km","7,8L/100km",589km,181g/km,1399kg (3084lb),https://www.guideautoweb.com/constructeurs/acu...


In [20]:
df_clean = df.copy()


# for index, row in df_clean.head().iterrows():
#     print(row)

df_clean = df_clean.iloc[0:300] 
df_clean = df_clean[['marque', 'modele', 'annee', 'eff_combi', 'eff_ges']] 

df_clean['eff'] = df_clean['eff_combi'].str.replace(',', '.',regex=True).str.replace('L/100km', '',regex=True).str.replace('Lₑ/100km', '', regex=True).str.replace('n.d.','0',regex=True).fillna(0).astype(float)
df_clean['ges'] = df_clean['eff_ges'].str.replace('g/km', '',regex=True).str.replace('n.d.','0',regex=True).fillna(0).astype(int)
df_clean = df_clean.drop(columns={'eff_combi', 'eff_ges'})

df_clean = df_clean.groupby(['marque', 'modele', 'annee']).mean().reset_index()
# pour tester, certain eff (probablement les PHEV on des valeurs entre parenthèse, à ajuster)

# Refaire rouler sur cette ordi là
# Créer une nouvelle colonne qui distingue le type de drivetrain (ESS, VEB, HYB, VHB)
# Faire un groupby sur 'marque', 'modele', 'annee' pour l'efficacité et les émissions 

df_clean.head()

,marque,modele,annee,eff,ges
0,acura,csx,2008,8.10,0.000000
1,acura,csx,2009,8.10,0.000000
2,acura,csx,2010,8.20,187.666667
3,acura,csx,2011,7.95,181.500000
4,acura,ilx,2013,7.18,165.400000


In [26]:
# Store dans une db
connection = sqlite3.connect('scrap_data.db')
cursor = connection.cursor()

df_clean.to_sql('vehicles', connection, if_exists='replace', index=False)

connection.commit()
connection.close()

In [27]:
# Accèder au db
connection = sqlite3.connect('scrap_data.db')
cursor = connection.cursor()

cursor.execute('SELECT * FROM vehicles')

# Columns name
column_names = [description[0] for description in cursor.description]
print("Column Names:", column_names)

# Values
rows = cursor.fetchall()
for row in rows:
    print(row)

connection.close()

Column Names: ['marque', 'modele', 'annee', 'eff', 'ges']
('acura', 'csx', '2008', 8.1, 0.0)
('acura', 'csx', '2009', 8.1, 0.0)
('acura', 'csx', '2010', 8.2, 187.66666666666666)
('acura', 'csx', '2011', 7.949999999999999, 181.5)
('acura', 'ilx', '2013', 7.18, 165.4)
('acura', 'ilx', '2014', 7.160000000000001, 27.2)
('acura', 'ilx', '2015', 7.2, 28.0)
('acura', 'ilx', '2016', 8.1, 189.0)
('acura', 'ilx', '2017', 8.2, 192.0)
('acura', 'ilx', '2018', 8.2, 192.0)
('acura', 'ilx', '2019', 8.6, 199.0)
('acura', 'ilx', '2020', 8.6, 199.0)
('acura', 'ilx', '2021', 8.6, 199.0)
('acura', 'ilx', '2022', 8.6, 200.0)
('acura', 'integra', '2023', 5.96, 138.4)
('acura', 'integra', '2024', 8.125, 188.75)
('acura', 'mdx', '2008', 12.1, 0.0)
('acura', 'mdx', '2009', 12.1, 0.0)
('acura', 'mdx', '2010', 11.6, 267.0)
('acura', 'mdx', '2011', 11.6, 267.0)
('acura', 'mdx', '2012', 11.6, 267.0)
('acura', 'mdx', '2013', 11.6, 267.0)
('acura', 'mdx', '2014', 0.0, 0.0)
('acura', 'mdx', '2015', 11.1, 255.0)
('acu